In [ ]:
!pip install keras-tuner

In [1]:
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
import os

In [2]:
import sys
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 0 = all logs, 1 = filter out INFO, 2 = filter out INFO and WARNING, 3 = ERROR only

# Increase recursion limit to prevent potential issues
sys.setrecursionlimit(100000)

In [3]:
from tensorflow import keras
from sklearn.model_selection import train_test_split

# Load the MNIST dataset
(x_all, y_all), _ = keras.datasets.mnist.load_data()

# Flatten and normalize the images
x_all = x_all.reshape((x_all.shape[0], -1)).astype("float32") / 255.0

# Split into train+val and test (80/20)
x_temp, x_test, y_temp, y_test = train_test_split(x_all, y_all, test_size=0.2, random_state=42)

# Split train+val into train and validation (75/25 of 80% = 60/20 overall)
x_train, x_val, y_train, y_val = train_test_split(x_temp, y_temp, test_size=0.25, random_state=42)

In [4]:
(x_train, y_train), (x_val, y_val) = mnist.load_data()
x_train, x_val = x_train / 255.0, x_val / 255.0

print(f'Training data shape: {x_train.shape}')
print(f'Validation data shape: {x_val.shape}')

Training data shape: (60000, 28, 28)
Validation data shape: (10000, 28, 28)


In [7]:
def build_model(hp):
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'),
        Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [8]:

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='my_dir',
    project_name='intro_to_kt'
)

# Display a summary of the search space 
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


/opt/homebrew/anaconda3/envs/colab/lib/python3.11/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val)) 

tuner.results_summary() 

Trial 10 Complete [00h 00m 50s]
val_accuracy: 0.9790999889373779

Best val_accuracy So Far: 0.9790999889373779
Total elapsed time: 00h 05m 20s
Results summary
Results in my_dir/intro_to_kt
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 09 summary
Hyperparameters:
units: 512
learning_rate: 0.002051026678741402
Score: 0.9790999889373779

Trial 03 summary
Hyperparameters:
units: 384
learning_rate: 0.0034070984533815926
Score: 0.9772000014781952

Trial 02 summary
Hyperparameters:
units: 160
learning_rate: 0.00046710694505239296
Score: 0.9760499894618988

Trial 00 summary
Hyperparameters:
units: 160
learning_rate: 0.004142803010919861
Score: 0.9741999804973602

Trial 07 summary
Hyperparameters:
units: 160
learning_rate: 0.004217818813601371
Score: 0.9737499952316284

Trial 01 summary
Hyperparameters:
units: 160
learning_rate: 0.00029825531558087945
Score: 0.9730000197887421

Trial 05 summary
Hyperparameters:
units: 352
learning_rate: 0.0001735701471314462
Scor

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0] 
print(f""" 

The optimal number of units in the first dense layer is {best_hps.get('units')}. 

The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}. 

""") 

# Step 2: Build and Train the Model with Best Hyperparameters 
model = tuner.hypermodel.build(best_hps) 
model.fit(x_train, y_train, epochs=10, validation_split=0.2) 

# Evaluate the model on the test set 
test_loss, test_acc = model.evaluate(x_val, y_val) 
print(f'Test accuracy: {test_acc}') 